In [98]:
import numpy as np
import random
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
%matplotlib inline

//anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:1357: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [99]:
from sklearn import gaussian_process
from sklearn.gaussian_process.kernels import ConstantKernel, RBF, WhiteKernel

In [100]:
training = np.genfromtxt('/Users/linggeli/trainingGP.csv', delimiter=',')
gradient = np.genfromtxt('/Users/linggeli/gradientGP.csv', delimiter=',')
energy = np.genfromtxt('/Users/linggeli/energyGP.csv', delimiter=',')
print(training.shape)
print(gradient.shape)
print(energy.shape)

(2917, 51)
(2917, 51)
(2917,)


In [22]:
from sklearn import preprocessing

In [23]:
scaler = preprocessing.StandardScaler().fit(training)
training_scaled = preprocessing.scale(training)

In [101]:
gp = gaussian_process.GaussianProcessRegressor(kernel=RBF(), normalize_y=True)
gp.fit(training, energy)

GaussianProcessRegressor(alpha=1e-10, copy_X_train=True,
             kernel=RBF(length_scale=1), n_restarts_optimizer=0,
             normalize_y=True, optimizer='fmin_l_bfgs_b',
             random_state=None)

In [102]:
gp.kernel_

RBF(length_scale=1e-05)

In [105]:
kernel_fitted = RBF(length_scale=0.1)
kernel_fitted.get_params()

{'length_scale': 0.1, 'length_scale_bounds': (1e-05, 100000.0)}

In [106]:
K_inverse = np.linalg.inv(kernel_fitted(training))

In [107]:
energy_normalized = (energy - np.mean(energy)) / np.std(energy)

In [108]:
def compute_gradient(x):
    n = K_inverse.shape[0]
    grad = np.zeros(51)
    for i in range(51):
        partial = kernel_fitted(x, training) * (training[:, i] - x[i]) / (10.0)**2
        grad[i] = np.dot(np.dot(partial, K_inverse), energy_normalized)
    return(grad)

In [109]:
grad_hat = np.zeros((100, 51))
for i in range(100):
    grad_hat[i, :] = compute_gradient(training[i, :])

In [110]:
print(np.mean(np.square(gradient[0:100, :] - grad_hat)))

3260.9338016
